<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Liberaries

In [ ]:
import numpy as np
import pickle
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas
import os
import cv2
import mediapipe as mp
import numpy as np
import time
import pickle
from math import cos, sin
from sklearn.preprocessing import MinMaxScaler,StandardScaler


## Reading the prepared dataset and split it

In [ ]:
path_local='../Dataset/Data/data468_normed_low.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/media_with_angles_low.csv'
data =pd.read_csv(path_local)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,yaw,pitch,roll,image_name
0,0.0,0.768221,0.042679,0.298753,0.896258,0.768221,0.256074,-2.987527,0.256074,0.384111,...,-22.0,-86.0,-83.0,-80.0,-91.0,-95.0,0.725284,-0.140406,-0.174674,image03195
1,0.0,-0.119313,-0.053028,-0.198854,-0.132570,-0.132570,-0.092799,-0.755647,-0.106056,-0.119313,...,-15.0,-74.0,-72.0,-70.0,-84.0,-86.0,-0.233745,-0.116700,0.095809,image03247
2,0.0,-0.257694,0.000000,-0.227377,-0.303170,-0.257694,-0.090951,-0.212219,-0.090951,-0.151585,...,-23.0,-76.0,-72.0,-70.0,-86.0,-92.0,-1.330352,0.003072,0.362599,image03214
3,0.0,-0.011573,0.023147,-0.081013,0.000000,0.000000,0.046293,-0.763836,0.057866,0.057866,...,-21.0,-105.0,-99.0,-94.0,-121.0,-125.0,-0.040907,-0.474087,0.019801,image03238
4,0.0,0.126856,0.063428,0.047571,0.158570,0.174427,0.206142,-0.856280,0.237856,0.269570,...,-24.0,-94.0,-91.0,-88.0,-99.0,-102.0,0.205732,-0.241359,-0.150232,image03236


In [ ]:
X = data.iloc[:,:-4]
X.head(1)

In [ ]:
y = data.iloc[:,-4:-1]
y.head(1)

,yaw,pitch,roll
0,0.725284,-0.140406,-0.174674


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.20, random_state=42, shuffle=True)

In [ ]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width /2
        tdy = height /2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img


## Load Trained Model

In [ ]:
pkl_filename='model_RM.pkl'
with open(pkl_filename, 'rb') as file:
    pretrained_model = pickle.load(file)

## Real Time Test

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

cap = cv2.VideoCapture(0)
i = 0
while cap.isOpened():
    success, image = cap.read()
    start = time.time()

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance
    image.flags.writeable = False
    # Get the result
    results = face_mesh.process(image)
    # To improve performance
    image.flags.writeable = True
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_2d, x_points,  y_points=[],[],[]
  
    if results.multi_face_landmarks:   
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 1 :
                    nose_2d = (lm.x * img_w, lm.y * img_h)
                    nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                elif idx == 428 :  chin_2d = (lm.x * img_w, lm.y * img_h)      
                x, y = int(lm.x * img_w), int(lm.y * img_h)
                x_points.append(x) 
                y_points.append(y) 
                face_2d.append([x, y])
                
        # Convert it to the NumPy array
        distX =   chin_2d[0]-nose_2d[0] 
        distY = chin_2d[1]  -nose_2d[1]
        dist = (distX**2 + distY**2)**0.5
        
        face_2d = np.array(face_2d, dtype=np.float64)
        x_point=np.array(np.transpose(x_points).astype('float'),dtype=np.float64).reshape(1,-1)
        y_point=np.array(np.transpose(y_points).astype('float'),dtype=np.float64).reshape(1,-1)
        x_point = (x_point-nose_2d[0])/dist
        y_point = (y_point-nose_2d[0])/dist
        new_data = np.concatenate((x_point,y_point),axis=1)
        

        # Load pretrained model
        angles = pretrained_model.predict(new_data) #yaw , pitch , roll
        print(angles)
        x = angles[0][0] #yaw
        y = angles[0][1] #pitch
        z = angles[0][2] #roll
              
        image = draw_axis(image,-x,y,-z,int(nose_2d[0]),int(nose_2d[1])) 
        # Add the text on the image      
        cv2.putText(image, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


        end = time.time()
        totalTime = end - start
        fps = 1 / totalTime
        print("FPS: ", fps)

        cv2.putText(image, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

        mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)


    cv2.imshow('Head Pose Estimation', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()

[[-0.33695568  0.01012494 -0.31025815]]
FPS:  1.006484080874757
[[-0.35241516  0.01190857 -0.15800328]]
FPS:  1.9314882525765125
[[-0.32442058  0.01175024 -0.21726985]]
FPS:  2.0143452867784672
[[-0.35230578  0.01099594 -0.2706532 ]]
FPS:  2.1632300243229694
[[-0.33269183  0.01099594 -0.25667607]]
FPS:  19.068831948207823
[[-0.34356897  0.01182943 -0.21544858]]
FPS:  18.677294526800466
[[-0.33343802  0.01099594 -0.24254772]]
FPS:  22.29696296335674
[[-0.33667148  0.01099594 -0.2272865 ]]
FPS:  21.746454921113894
[[-0.36230103  0.00242731 -0.28438521]]
FPS:  20.49901764332144
[[-0.34788456 -0.03691434 -0.15778635]]
FPS:  21.754575961742937
[[-0.35735804 -0.07531213 -0.08790109]]
FPS:  19.650145936500053
[[-0.38182972 -0.07393324 -0.3711019 ]]
FPS:  21.809665442973472
[[-0.35452045 -0.07202139 -0.17199953]]
FPS:  19.231986794442662
[[-0.35983385 -0.07080327 -0.39728218]]
FPS:  21.795612093245616
[[-0.38740555 -0.06765938 -0.50194226]]
FPS:  21.811706951782668
[[-0.3811209  -0.06756946 -0

[[-0.39262025 -0.07308042 -0.08425238]]
FPS:  20.853489250840244
[[-0.40251831 -0.07496753 -0.06275645]]
FPS:  21.723140667081
[[-0.39755652 -0.07496753 -0.06547248]]
FPS:  21.060712115810457
[[-0.39418252 -0.07686355 -0.04273354]]
FPS:  19.314526750108215
[[-0.39287336 -0.07496753 -0.06547248]]
FPS:  21.34766587267656
[[-0.39015873 -0.06925665 -0.06091625]]
FPS:  22.803557836591782
[[-0.40412211 -0.07960492 -0.20979195]]
FPS:  21.432643321052442
[[-0.40513444 -0.08112145 -0.22202645]]
FPS:  20.916405272107994
[[-0.42500065 -0.07055273 -0.14575541]]
FPS:  20.053280295279166
[[-0.37979072 -0.07346508 -0.18319743]]
FPS:  21.667367507503474
[[-0.39184575 -0.07346508 -0.26584781]]
FPS:  22.29696296335674
[[-0.38107515 -0.0215074  -0.33233777]]
FPS:  19.73112295540826
[[-0.37792011 -0.02333272 -0.44333031]]
FPS:  21.414915832307937
[[-0.35668294 -0.02206541 -0.43280869]]
FPS:  20.904729389600227
[[-0.39031075 -0.02540161 -0.5134182 ]]
FPS:  22.147555180061254
[[-0.33443149 -0.04107289 -0.62

FPS:  22.10634944896249
[[-0.52300046 -0.06834723 -0.44132256]]
FPS:  21.810686149606095
[[-0.50981417 -0.06235237 -0.40178488]]
FPS:  16.272507895124807
[[-0.47152988 -0.07043358 -0.29981587]]
FPS:  21.900312242191333
[[-0.4912113  -0.07088044 -0.64692225]]
FPS:  21.15119943923631
[[-0.50497091 -0.07612067 -0.2969474 ]]
FPS:  22.590344054979855
[[-0.38180944 -0.07088044 -0.27707205]]
FPS:  21.085805063443864
[[-0.35390747 -0.07893896 -0.29681087]]
FPS:  21.668151056465362
[[-0.39663211 -0.07555644 -0.14201588]]
FPS:  21.02313178854087
[[-0.36645269 -0.08199518  0.04303906]]
FPS:  21.83100673516338
[[-0.39128448 -0.06069771 -0.30867161]]
FPS:  23.23971631205674
[[-0.37652931 -0.06085205 -0.00341224]]
FPS:  22.88093393704653
[[-0.35647924 -0.06085205 -0.25080978]]
FPS:  19.5902140101447
[[-0.35746359 -0.06085205 -0.25080978]]
FPS:  20.369598368219126
[[-0.3862597  -0.06090141 -0.26580204]]
FPS:  20.903270820770185
[[-0.35707464 -0.06129891 -0.19815576]]
FPS:  20.129696108732794
[[-0.350

[[-0.62236794 -0.20794103  0.59739363]]
FPS:  22.295659199880927
[[-0.50550815 -0.07269877  0.54472946]]
FPS:  22.719440126101627
[[-0.53007169 -0.07269877  0.58508088]]
FPS:  22.804921678329283
[[-0.51512087 -0.07532047  0.58664845]]
FPS:  21.370507372647324
[[-0.50011313 -0.1241301   0.61618698]]
FPS:  22.286537122939016
[[-0.50965175 -0.07269877  0.57760006]]
FPS:  22.801574357970733
[[-0.50103278 -0.11857096  0.61618698]]
FPS:  20.120329462105623
[[-0.50011313 -0.15616385  0.6286755 ]]
FPS:  22.70369869168187
[[-0.53461011 -0.07532047  0.5822653 ]]
FPS:  22.72855060447927
[[-0.51338608 -0.07269877  0.63239435]]
FPS:  22.673506786963408
[[-0.50011313 -0.07269877  0.5887786 ]]
FPS:  21.97167043835387
[[-0.50011313 -0.07269877  0.5935656 ]]
FPS:  23.333318498408953
[[-0.51580774 -0.07269877  0.58835664]]
FPS:  22.53972109519843
[[-0.50103278 -0.07269877  0.56507122]]
FPS:  21.61299365157886
[[-0.51759519 -0.07269877  0.57116256]]
FPS:  22.34233389620144
[[-0.55994191 -0.07269877  0.52

FPS:  21.1138271952963
[[-0.36257887 -0.08048991  0.03559031]]
FPS:  21.307864643396005
[[-0.36060948 -0.06336823 -0.06894108]]
FPS:  22.980850679129706
[[-0.35978898 -0.08048991  0.05476849]]
FPS:  21.576525782954032
[[-0.37751847 -0.07842302  0.0103043 ]]
FPS:  22.802690022235634
[[-0.34484595 -0.07482452 -0.00132254]]
FPS:  21.840100809181134
[[-0.35277432 -0.08048991  0.03777618]]
FPS:  18.23539078905608
[[-0.34566933 -0.06898089 -0.16857113]]
FPS:  21.60531182230739
[[-0.35110309 -0.07867927  0.00656024]]
FPS:  22.803309882892776
[[-0.34939304 -0.08048991 -0.06297072]]
FPS:  20.66078184110971
[[-0.35765058 -0.06996269 -0.07612859]]
FPS:  21.96959887698126
[[-0.35636423 -0.08048991  0.05002314]]
FPS:  22.398769598837955
[[-0.34566933 -0.07343956 -0.04382974]]
FPS:  22.650976664812525
[[-0.35509061 -0.06705812 -0.08273426]]
FPS:  22.180583612729908
[[-0.35695734 -0.06872924 -0.03385405]]
FPS:  22.7727290002769
[[-0.35251782 -0.06510838 -0.07651517]]
FPS:  21.28569688603792
[[-0.3588

[[-0.39839162 -0.12606693  0.09086129]]
FPS:  17.28099146313326
[[-0.18345018 -0.10468568  0.07996075]]
FPS:  18.068226952187718
[[-0.23929448 -0.09257031  0.09086129]]
FPS:  14.928048745764642
[[-0.25301989 -0.1268312   0.09950937]]
FPS:  19.48953570499238
[[-0.34763893 -0.12964521  0.10735998]]
FPS:  17.20888363715438
[[-0.2237871  -0.12816166  0.08351733]]
FPS:  17.144309964969935
[[-0.1072328  -0.14156743  0.08502795]]
FPS:  16.783056503663275
[[-0.44420795 -0.12725427  0.06716196]]
FPS:  20.218190231957273
[[-0.1840743  -0.12534765  0.06817866]]
FPS:  18.643588341712118
[[-0.19437903 -0.13173994  0.08264211]]
FPS:  20.706988225420258
[[-0.38706791 -0.12964521  0.09253155]]
FPS:  20.55044145459534
[[-0.4553185  -0.12379804  0.07714229]]
FPS:  20.42534624150223
[[-0.35866005 -0.10546837  0.07872556]]
FPS:  17.18716424148797
[[-0.39903371 -0.12606693  0.08268108]]
FPS:  18.33775931795825
[[-0.47121373 -0.12606693  0.07679536]]
FPS:  18.04692548975737
[[-0.36655053 -0.12606693  0.0921

FPS:  16.918314744973074
[[-0.53666507 -0.10267871 -0.90457981]]
FPS:  20.00364369959509
[[-0.58123828 -0.15008047 -0.35235169]]
FPS:  17.65814556599643
[[-0.43724391 -0.07975983 -0.00069608]]
FPS:  9.228736828440228
[[-0.44275413 -0.05682397  0.01460101]]
FPS:  5.082569407670318
[[-0.3928848  -0.0569512   0.00773869]]
FPS:  4.177111011078401
[[-0.38553395 -0.07411395  0.03277628]]
FPS:  4.802304572735787
[[-0.51968621 -0.1165099  -0.04209437]]
FPS:  4.191152253505364
[[-0.94989048 -0.12870278 -0.06164808]]
FPS:  5.413723581936976
[[-0.93730782 -0.10437205 -0.04615209]]
FPS:  4.218535266571185
[[-0.43484721 -0.12311914 -0.04418217]]
FPS:  4.26944190814138
[[-0.4529713  -0.09293716  0.0288101 ]]
FPS:  4.229207650741571
[[-0.41556792 -0.06944043 -0.0529631 ]]
FPS:  4.729050887168333
[[-0.39588768 -0.07008056 -0.00556525]]
FPS:  4.48316796124703
[[-0.36662721 -0.07459823 -0.0051516 ]]
FPS:  4.224504986145957
[[-0.55100462 -0.07475033 -0.02798556]]
FPS:  4.263140936704975
[[-0.60301626 -0.

[[-0.41974831 -0.07801948  0.04485599]]
FPS:  18.819002494660708
[[-0.42795435 -0.08445034  0.05534043]]
FPS:  21.606870012724155
[[-0.40575966 -0.08691287  0.07690658]]
FPS:  14.052069967133807
[[-0.43212176 -0.0857382   0.06808924]]
FPS:  19.319864761536266
[[-0.39645284 -0.0870419   0.04831309]]
FPS:  20.47450147665422
[[-0.40218213 -0.07945805  0.06150471]]
FPS:  16.764340986122658
[[-0.5607544  -0.08163633  0.08278288]]
FPS:  22.052187445780472
[[-0.56713174 -0.08691287  0.06001186]]
FPS:  20.4299227479518
[[-0.475306   -0.07894616  0.08029814]]
FPS:  18.565439093484418
[[-0.61461787 -0.08691287  0.07647961]]
FPS:  17.81996932502305
[[-0.57727714 -0.08445034  0.06635839]]
FPS:  15.867320380122269
[[-0.42693638 -0.0857382   0.06442512]]
FPS:  20.497414795773754
[[-0.48224658 -0.08691287  0.06663024]]
FPS:  15.660438789073584
[[-0.40691023 -0.08294516  0.07978906]]
FPS:  20.457923822437703
[[-0.41202515 -0.08445034  0.07930479]]
FPS:  19.271928615407237
[[-0.45359165 -0.102576    0.

[[-0.36922308 -0.06579735 -0.00745383]]
FPS:  20.065943949556516
[[-0.38138456 -0.0835676   0.04548719]]
FPS:  16.127348370079286
[[-0.38461228 -0.06432797  0.04699021]]
FPS:  19.59286785348923
[[-0.37719655 -0.06349898  0.02170441]]
FPS:  18.48973528180035
[[-0.37282517 -0.06755285  0.05130375]]
FPS:  21.928478815508804
[[-0.37481955 -0.05900531  0.03794582]]
FPS:  17.603968790266055
[[-0.36348582 -0.05670694  0.03808916]]
FPS:  16.16003205572765
[[-0.39400527 -0.06166938  0.04789851]]
FPS:  20.901083349113488
[[-0.36739073 -0.06579735  0.05156263]]
FPS:  18.141610221541708
[[-0.35891158 -0.04720338  0.08530414]]
FPS:  20.017773196073097
[[-0.38937314 -0.07112345  0.04745207]]
FPS:  19.24230982736394
[[-0.38569781 -0.05670694  0.04789851]]
FPS:  17.818228162145168
[[-0.40258866 -0.07138901  0.06849223]]
FPS:  19.63993257164263
[[-0.40743164 -0.06579735  0.01591317]]
FPS:  17.941857878616773
[[-0.38461228 -0.05670694  0.01591317]]
FPS:  17.52672904069634
[[-0.37345597 -0.07181312  0.04

FPS:  19.162485551509725
[[-0.36528946  0.02495745 -0.26543994]]
FPS:  21.422353427889945
[[-0.33983474  0.02495745 -0.16369342]]
FPS:  20.06699997607827
[[-0.34359322  0.02495745 -0.2350778 ]]
FPS:  19.816795335783873
[[-0.35330854  0.02495745 -0.26504042]]
FPS:  22.29625180074103
[[-0.33851941  0.02495745 -0.20594615]]
FPS:  18.026216488022072
[[-0.35230578  0.02495745 -0.26070689]]
FPS:  18.16919431484923
[[-0.34617037  0.02495745 -0.20594615]]
FPS:  19.66479441136481
[[-0.34712295  0.02495745 -0.22924649]]
FPS:  19.99506118693598
[[-0.35201471  0.02495745 -0.26504042]]
FPS:  21.7904033582012
[[-0.33983474  0.02495745 -0.19571288]]
FPS:  19.234368049600345
[[-0.33983474  0.02495745 -0.23865208]]
FPS:  19.29542307462288
[[-0.34712295  0.02495745 -0.26504042]]
FPS:  18.58230068892187
[[-0.35230578  0.02495745 -0.20836539]]
FPS:  19.00971718636693
[[-0.35230578  0.02495745 -0.20836539]]
FPS:  18.81360007176819
[[-0.33251608  0.02495745 -0.22225963]]
FPS:  17.057504920859564
[[-0.352305